In [31]:
import sys
sys.path.insert(0, '../..')

In [32]:
%matplotlib inline
import numpy as np
import torch
import matplotlib.pyplot as plt
from tqdm import tnrange, tqdm_notebook

from causal_meta.utils.data_utils import generate_data_categorical
from causal_meta.bivariate.categorical1 import StructuralModel

In [33]:
N = 10
model = StructuralModel(N)#, dtype=torch.float64)

In [34]:
num_episodes = 500
batch_size = 100 # 1
num_test = 10000
num_training = 10 # 100
num_transfers = 10 # 100

In [35]:
optimizer = torch.optim.SGD(model.modules_parameters(), lr=1.)

In [36]:
losses = np.zeros((2, num_training, num_transfers, num_episodes))

for k in tnrange(num_training):
    
    #Marginal Probability
    pi_A_1 = np.random.dirichlet(np.ones(N))
    pi_B_1 = np.random.dirichlet(np.ones(N))
    pi_C_1 = np.random.dirichlet(np.ones(N))
    
    #Conditional Probability
    pi_B_A = np.random.dirichlet(np.ones(N), size=N)
    pi_B_A = np.random.dirichlet(np.ones(N), size=N)
    pi_B_A = np.random.dirichlet(np.ones(N), size=N)
    
    for j in tnrange(num_transfers, leave=False):
        #model.setGroundTruth(pi_A_1, pi_B_A)
        pi_A_2 = np.random.dirichlet(np.ones(N))
        pi_B_2 = np.random.dirichlet(np.ones(N))
        #pi_C_2 = np.random.dirichlet(np.ones(N))
        
        x_val1 = torch.from_numpy(generate_data_categorical(num_test, pi_A_2, pi_B_A))
        x_val2 = torch.from_numpy(generate_data_categorical(num_test, pi_B_2, pi_C_B))
        #x_val3 = torch.from_numpy(generate_data_categorical(num_test, pi_C_2, pi_A_C))
        
        
        for i in range(num_episodes):
            x_transfer1 = torch.from_numpy(generate_data_categorical(batch_size, pi_A_2, pi_B_A))
            x_transfer2 = torch.from_numpy(generate_data_categorical(batch_size, pi_B_2, pi_C_B))
            
            model.zero_grad()
            
            loss_A_B = -torch.mean(model.model_A_B_C(x_transfer1))
            loss_B_A = -torch.mean(model.model_C_B_A(x_transfer1))
            
            #loss_B_C = -torch.mean(model.model_B_C(x_transfer2))
            #loss_C_B = -torch.mean(model.model_C_B(x_transfer2))
            
            loss1 = loss_A_B + loss_B_A
            #loss2 = loss_B_C + loss_C_B
            
            with torch.no_grad():
                val_loss_A_B_C = -torch.mean(model.model_A_B_C(x_val1))
                val_loss_C_B_A = -torch.mean(model.model_C_B_A(x_val1))
                
                #val_loss_B_C = -torch.mean(model.model_B_C(x_val2))
                #val_loss_C_B = -torch.mean(model.model_C_B(x_val2))

            losses[:, k, j, i] = [val_loss_A_B_C.item(), val_loss_C_B_A.item()]

            loss1.backward()
            #loss2.backward()
            optimizer.step()

<ipython-input-36-8f89b7afc730>:3: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for k in tnrange(num_training):


<ipython-input-36-8f89b7afc730>:15: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for j in tnrange(num_transfers, leave=False):


AttributeError: 'StructuralModel' object has no attribute 'model_A_B_C'

In [ ]:
flat_losses = -losses.reshape((2, -1, num_episodes))
losses_25, losses_50, losses_75 = np.percentile(flat_losses, (25, 50, 75), axis=1)

plt.figure(figsize=(9, 5))

ax = plt.subplot(1, 1, 1)
ax.plot(losses_50[0], color='C0', label=r'$A \rightarrow B$', lw=2)
ax.fill_between(np.arange(num_episodes), losses_25[0], losses_75[0], color='C0', alpha=0.2)
ax.plot(losses_50[1], color='C3', label=r'$B \rightarrow A$', lw=2)
ax.fill_between(np.arange(num_episodes), losses_25[1], losses_75[1], color='C3', alpha=0.2)
ax.set_xlim([0, flat_losses.shape[1] - 1])
ax.tick_params(axis='both', which='major', labelsize=13)
ax.legend(loc=4, prop={'size': 13})
ax.set_xlabel('Number of examples', fontsize=14)
ax.set_ylabel(r'$\log P(D\mid \cdot \rightarrow \cdot)$', fontsize=14)

plt.show()